In [2]:
import pandas as pd

train = pd.read_csv('train.csv')
print("Training Set:"% train.columns, train.shape, len(train))
test = pd.read_csv('test.csv')
print("Test Set:"% test.columns, test.shape, len(test))

Training Set: (31962, 3) 31962
Test Set: (17197, 2) 17197


In [3]:
import re
def  clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df
test_clean = clean_text(test, "tweet")
train_clean = clean_text(train, "tweet")

In [6]:
pip install scikit-learn


     ---------------------------------------- 0.0/9.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/9.2 MB 991.0 kB/s eta 0:00:10
      --------------------------------------- 0.2/9.2 MB 2.8 MB/s eta 0:00:04
     -- ------------------------------------- 0.6/9.2 MB 4.9 MB/s eta 0:00:02
     ------ --------------------------------- 1.5/9.2 MB 8.4 MB/s eta 0:00:01
     ---------- ----------------------------- 2.5/9.2 MB 11.3 MB/s eta 0:00:01
     --------------- ------------------------ 3.7/9.2 MB 13.7 MB/s eta 0:00:01
     -------------------- ------------------- 4.8/9.2 MB 15.4 MB/s eta 0:00:01
     -------------------------- ------------- 6.0/9.2 MB 16.7 MB/s eta 0:00:01
     ------------------------------ --------- 7.1/9.2 MB 17.6 MB/s eta 0:00:01
     ----------------------------------- ---- 8.3/9.2 MB 18.3 MB/s eta 0:00:01
     ---------------------------------------  9.2/9.2 MB 18.5 MB/s eta 0:00:01
     ---------------------------------------- 9.2/9.2 MB 17.9 


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\ayush\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [7]:
import sklearn
print(sklearn.__version__)


1.3.0


In [8]:
from sklearn.utils import resample
train_majority = train_clean[train_clean.label==0]
train_minority = train_clean[train_clean.label==1]
train_minority_upsampled = resample(train_minority, 
                                 replace=True,    
                                 n_samples=len(train_majority),   
                                 random_state=123)
train_upsampled = pd.concat([train_minority_upsampled, train_majority])
train_upsampled['label'].value_counts()

label
1    29720
0    29720
Name: count, dtype: int64

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('nb', SGDClassifier()),])

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'],
                                                    train_upsampled['label'],random_state = 0)

In [11]:
model = pipeline_sgd.fit(X_train, y_train)
y_predict = model.predict(X_test)
from sklearn.metrics import f1_score
f1_score(y_test, y_predict)

0.9696